In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torchsummary import summary

In [2]:
path = './Data/trainset/trainset/Cat/'
list_images = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

In [253]:
## mean and standard deviation of the ImageNet was: ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# normalize = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))


normalize = transforms.Normalize((0.4895832, 0.4546405, 0.41594946), 
                                 (0.2520022, 0.24522494, 0.24728711))
transform = transforms.Compose([transforms.ToTensor(),
                               normalize])
traindata = torchvision.datasets.ImageFolder('./Data/trainset/trainset/', transform=transform)
testset =torchvision.datasets.ImageFolder('./Data/testset/', transform=transform)


size = [int(len(traindata)*0.9), len(traindata) - int(len(traindata)*0.9)]

trainset, validset = torch.utils.data.random_split(dataset=traindata, lengths=size)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=128,
                                          shuffle=True)

validloader = torch.utils.data.DataLoader(validset,
                                          batch_size=128,
                                          shuffle=True)

testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=512,
                                        shuffle=True)
classes = ('Cat', 'Dog')

In [248]:
cuda_available = torch.cuda.is_available()
print (cuda_available)

True


In [251]:
## get the mean and variance of our dataset for normalization

train = trainloader.__iter__().next()[0]
print (len(trainloader.dataset))
print('Mean: {}'.format(np.mean(train.numpy(), axis=(0, 2, 3))))
print('STD: {}'.format(np.std(train.numpy(), axis=(0, 2, 3))))

17998
Mean: [0.4895832  0.4546405  0.41594946]
STD: [0.2520022  0.24522494 0.24728711]


In [261]:
a = trainloader.dataset

In [262]:
def init_weights(m):
    if type(m) == nn.Linear:
        print('man injam')
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [263]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=3, out_channels=16,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(16),
            nn.ReLU(),
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(64),
            nn.ReLU(),
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 5
            nn.Conv2d(in_channels=128, out_channels=128,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 6
            nn.Conv2d(in_channels=128, out_channels=256,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
             # Layer 7
            nn.Conv2d(in_channels=256, out_channels=256,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            # Layer 8
            nn.Conv2d(in_channels=256, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
            nn.ReLU(),
            # Layer 9
            nn.Conv2d(in_channels=512, out_channels=512,kernel_size=(3,3) , padding=1),
#             nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            
        )
        self.net = nn.Linear(512, 2)
        self.soft = nn.Softmax()
        self.init_weights()
    def forward(self, x):
        out = self.net(self.conv(x).squeeze())
        return self.soft(out)
    
    def init_weights(self):
#         if type(self.conv) == nn.Linear:
#         print('man injam conv')
#         torch.nn.init.xavier_uniform(self.conv.weight)
        for m in self.conv:
            if isinstance(m,nn.Conv2d):
                nn.init.xavier_uniform(m.weight)
                m.bias.data.fill_(0.01)

        if type(self.net) == nn.Linear:
            nn.init.xavier_uniform(self.net.weight)
            self.net.bias.data.fill_(0.01)

In [264]:
net = NNet()
# net.apply(init_weights)
learning_rate = 0.01
if cuda_available:
    net.cuda()
optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate, momentum=0.95)
# optimizer = torch.optim.Adam(net.parameters())
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:62: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:66: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [160]:
summary(net, (3,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             448
              ReLU-2           [-1, 16, 64, 64]               0
            Conv2d-3           [-1, 32, 64, 64]           4,640
              ReLU-4           [-1, 32, 64, 64]               0
         MaxPool2d-5           [-1, 32, 32, 32]               0
            Conv2d-6           [-1, 64, 32, 32]          18,496
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8          [-1, 128, 32, 32]          73,856
              ReLU-9          [-1, 128, 32, 32]               0
        MaxPool2d-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
             ReLU-12          [-1, 128, 16, 16]               0
        MaxPool2d-13            [-1, 128, 8, 8]               0
           Conv2d-14            [-1, 25

In [265]:
def learning_rate_decay(epoch, loss, lr):
    mean_loss = np.mean(loss[epoch-10:epoch-1])
    if np.float(loss[epoch]) < np.float(mean_loss):
        lr *= 0.1
    return lr

In [266]:
c = 0
acc_val = []
for epoch in range(30):
    losses = []
    # Train
    net.train()
    for batch_idx, (images, labels) in enumerate(trainloader):
        if cuda_available:
            images, labels = images.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.item())

        
        if batch_idx%30==0:
            print('Epoch: %d Train Loss : %.3f ' % (epoch, np.mean(losses)))
#             print('Epoch: %d Train Loss : %.3f ' % (epoch, loss.data.item()))
    
    # Evaluate
    net.eval()
    total = 0
    correct = 0
    for batch_idx, (images, labels) in enumerate(validloader):
        if cuda_available:
            images, targets = images.cuda(), labels.cuda()

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        accuracy = 100.*(correct.numpy()/total)
        acc_val.append(accuracy)

    print('Epoch: %d Validation Acc : %.3f' % (epoch, 100.*correct/total))
    print (learning_rate)
    print('--------------------------------------------------------------')
    net.train()
    c += 1
    if (c%10 == 0):
        learning_rate = learning_rate_decay(epoch, acc_val, learning_rate)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0 Train Loss : 0.694 
Epoch: 0 Train Loss : 0.693 
Epoch: 0 Train Loss : 0.693 
Epoch: 0 Train Loss : 0.692 
Epoch: 0 Train Loss : 0.690 
Epoch: 0 Validation Acc : 55.000
0.01
--------------------------------------------------------------
Epoch: 1 Train Loss : 0.670 
Epoch: 1 Train Loss : 0.663 
Epoch: 1 Train Loss : 0.654 
Epoch: 1 Train Loss : 0.654 
Epoch: 1 Train Loss : 0.646 
Epoch: 1 Validation Acc : 68.000
0.01
--------------------------------------------------------------
Epoch: 2 Train Loss : 0.566 
Epoch: 2 Train Loss : 0.599 
Epoch: 2 Train Loss : 0.605 
Epoch: 2 Train Loss : 0.603 
Epoch: 2 Train Loss : 0.603 
Epoch: 2 Validation Acc : 70.000
0.01
--------------------------------------------------------------
Epoch: 3 Train Loss : 0.553 
Epoch: 3 Train Loss : 0.576 
Epoch: 3 Train Loss : 0.574 
Epoch: 3 Train Loss : 0.574 
Epoch: 3 Train Loss : 0.573 
Epoch: 3 Validation Acc : 73.000
0.01
--------------------------------------------------------------
Epoch: 4 Train L

/home/laya/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/laya/.local/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch: 10 Train Loss : 0.449 
Epoch: 10 Train Loss : 0.444 
Epoch: 10 Train Loss : 0.449 
Epoch: 10 Train Loss : 0.453 
Epoch: 10 Validation Acc : 79.000
0.01
--------------------------------------------------------------
Epoch: 11 Train Loss : 0.446 
Epoch: 11 Train Loss : 0.453 
Epoch: 11 Train Loss : 0.448 
Epoch: 11 Train Loss : 0.446 
Epoch: 11 Train Loss : 0.443 
Epoch: 11 Validation Acc : 79.000
0.01
--------------------------------------------------------------
Epoch: 12 Train Loss : 0.443 
Epoch: 12 Train Loss : 0.434 
Epoch: 12 Train Loss : 0.429 
Epoch: 12 Train Loss : 0.424 
Epoch: 12 Train Loss : 0.425 
Epoch: 12 Validation Acc : 72.000
0.01
--------------------------------------------------------------
Epoch: 13 Train Loss : 0.576 
Epoch: 13 Train Loss : 0.451 
Epoch: 13 Train Loss : 0.438 
Epoch: 13 Train Loss : 0.427 
Epoch: 13 Train Loss : 0.423 
Epoch: 13 Validation Acc : 80.000
0.01
--------------------------------------------------------------
Epoch: 14 Train Loss :

In [267]:
torch.save(net.state_dict(), './model/SGD')

In [274]:
a = np.sort(acc_val)
for i in range(len(a)):
    print (a[i])
np.max(acc_val)
# print (losses)

51.953125
52.604166666666664
52.734375
52.96875
53.90625
54.6875
54.78515625
54.921875
54.947916666666664
54.97159090909091
55.12152777777778
55.208333333333336
55.24553571428571
55.30133928571429
55.40865384615385
55.45
67.96875
68.54166666666667
68.56971153846155
68.61979166666666
68.75
68.75
68.82102272727273
68.90625
68.9453125
68.95
69.0625
69.140625
69.35763888888889
69.53125
69.53125
69.921875
70.1
70.26041666666667
70.3125
70.3125
70.36830357142857
70.44270833333334
70.57291666666666
70.703125
70.75892857142857
70.83333333333334
70.8984375
70.8984375
70.9375
71.00694444444444
71.015625
71.09375
71.09375
71.25
71.484375
71.54017857142857
71.5625
71.61458333333334
71.61458333333334
71.6796875
71.6796875
71.6796875
71.70138888888889
71.875
71.875
71.875
71.875
71.93080357142857
72.08333333333333
72.1
72.109375
72.265625
72.265625
72.30902777777779
72.33072916666666
72.35576923076923
72.39583333333334
72.52604166666666
72.5360576923077
72.54464285714286
72.65625
72.65625
72.65625
7

88.28125

In [47]:
# torch.save(net.state_dict(), 
#            os.path.join('saved_model', 'cnnT1_epoch_{}_iter_{}_loss_{}_acc_{}_{}.t7'.format(epoch + 1, i,
#            last_loss, acc, datetime.datetime.now().strftime("%b_%d_%H:%M:%S"))))

In [55]:
# np_image = trainset[0
# img = torch.from_numpy(np_image)
# img = img.view(img.shape[1], img.shape[2], img.shape[0])
# # img = img / 2+0.5
# # npimg = img.numpy()
# # plt.imshow(np.transpose(npimg, (1, 2, 0)))
# plt.imshow(img)
# plt.show()